n_segments=8

In [1]:
import os
import av
import torch
import numpy as np
import torch.nn.functional as F
import random
import json
import time
import subprocess
import psutil
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoImageProcessor, TimesformerForVideoClassification
from collections import defaultdict
from tqdm.notebook import tqdm
from itertools import combinations

class VideoProcessor:
    def __init__(self, model_name, image_processor_name, device='cuda'):
        self.model = self.load_model(model_name)
        self.image_processor = AutoImageProcessor.from_pretrained(image_processor_name)
        self.device = device
        self.model.to(device)

    def load_model(self, model_name):
        if "timesformer" in model_name.lower():
            return TimesformerForVideoClassification.from_pretrained(model_name)
        else:
            raise ValueError(f"Unsupported model name: {model_name}")

    def split_video_into_segments(self, container, n_segments=8, frames_per_segment=16):
        frame_list = [frame.to_image() for frame in container.decode(video=0)]
        total_frames = len(frame_list)
        segment_length = total_frames // n_segments
        segments = []
        for i in range(n_segments):
            start = i * segment_length
            end = min(start + segment_length, total_frames)
            segment_frames = frame_list[start:end] if end - start == segment_length else frame_list[start:] + [frame_list[-1]] * (segment_length - (end - start))
            segments.append(segment_frames[:frames_per_segment])
        return segments

    def predict_video_and_segments(self, container, true_label):
        video_segments = self.split_video_into_segments(container)
        segment_outputs = []
        with torch.no_grad():
            for segment in video_segments:
                inputs = self.image_processor(list(segment), return_tensors="pt")
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
                try:
                    outputs = self.model(**inputs)
                    logits = outputs.logits
                    probabilities = F.softmax(logits, dim=-1)
                    pred_label = logits.argmax(-1).item()
                    pred_score = probabilities[0, pred_label].item()
                    segment_outputs.append((pred_label, pred_score, probabilities))  # Ensure this returns a tuple
                except RuntimeError as e:
                    print(f"Error processing segment: {e}")
                    continue
        return segment_outputs


class TemporalShap:
    def __init__(self, num_samples=100):
        self.num_samples = num_samples

    def approximate_shapley_values(self, segment_outputs, label_index):
        n = len(segment_outputs)
        shapley_values = [0] * n
        for _ in range(self.num_samples):
            random_subset = sorted(range(n), key=lambda _: random.random())
            subset_prob = torch.zeros_like(segment_outputs[0][2])
            for i, index in enumerate(random_subset):
                old_contribution = subset_prob[0, label_index].item()
                subset_prob += segment_outputs[index][2]
                subset_prob /= (i + 1)
                new_contribution = subset_prob[0, label_index].item()
                shapley_values[index] += new_contribution - old_contribution
        return [val / self.num_samples for val in shapley_values]

    def exact_shapley_values(self, segment_outputs, label_index):
        n = len(segment_outputs)
        shapley_values = [0] * n
        all_indices = list(range(n))
        for i in all_indices:
            marginal_contributions = []
            for subset_size in range(n):
                subsets = list(combinations([x for x in all_indices if x != i], subset_size))
                for subset in subsets:
                    subset_prob = torch.zeros_like(segment_outputs[0][2])
                    if subset:
                        subset_prob = torch.mean(torch.stack([segment_outputs[j][2] for j in subset]), dim=0)
                    with_i_prob = (subset_prob * len(subset) + segment_outputs[i][2]) / (len(subset) + 1)
                    marginal_contributions.append(with_i_prob[0, label_index].item() - subset_prob[0, label_index].item())
            shapley_values[i] = np.mean(marginal_contributions)
        return shapley_values

def get_gpu_energy():
    try:
        result = subprocess.check_output(['nvidia-smi', '--query-gpu=energy.draw', '--format=csv,noheader,nounits'])
        return float(result.strip().split()[0])
    except Exception as e:
        print(f"Error reading GPU energy: {e}")
        return 0.0

def get_cpu_energy():
    try:
        energy = 0.0
        for domain in psutil.sensors_battery():
            energy += domain.power * domain.energy
        return energy
    except Exception as e:
        print(f"Error reading CPU energy: {e}")
        return 0.0
    
import subprocess
import time

def get_gpu_power_draw():
    try:
        # This command retrieves the current power usage in watts.
        result = subprocess.check_output(['nvidia-smi', '--query-gpu=power.draw', '--format=csv,noheader,nounits'], text=True)
        return float(result.strip())
    except Exception as e:
        print(f"Error reading GPU power: {e}")
        return 0.0

def get_system_time():
    return time.time()  # Return the current system time in seconds

# Example usage in your processing function
start_time = get_system_time()
initial_gpu_power = get_gpu_power_draw()

# Execute your long-running process here
time.sleep(10)  # Simulating a delay

end_time = get_system_time()
final_gpu_power = get_gpu_power_draw()

time_consumed = end_time - start_time
average_gpu_power = (initial_gpu_power + final_gpu_power) / 2
energy_consumed = average_gpu_power * (time_consumed / 3600)  # Convert power usage in watts to kilowatt-hours if needed

def process_videos(video_processor, shap_calculator, sampled_files, true_labels, use_exact=False, start_index=0):
    predictions = []
    for idx, (video_file, true_label) in tqdm(enumerate(zip(sampled_files, true_labels)), desc="Processing videos", total=len(sampled_files), initial=start_index, unit="video"):
        if idx < start_index:
            continue
        file_path = os.path.join(config["video_directory"], video_file)
        container = av.open(file_path)
        try:
            segment_outputs = video_processor.predict_video_and_segments(container, true_label)
            if not segment_outputs:
                print(f"Skipping video {video_file} due to empty segment outputs.")
                continue
            video_probs = torch.mean(torch.stack([output[2] for output in segment_outputs]), dim=0)
            video_pred_label = video_probs.argmax().item()
            video_pred_score = video_probs[0, video_pred_label].item()
            video_true_score = video_probs[0, true_label].item()
            
            if use_exact:
                sv_true_label = shap_calculator.exact_shapley_values(segment_outputs, true_label)
                sv_video_pred = shap_calculator.exact_shapley_values(segment_outputs, video_pred_label)
            else:
                sv_true_label = shap_calculator.approximate_shapley_values(segment_outputs, true_label)
                sv_video_pred = shap_calculator.approximate_shapley_values(segment_outputs, video_pred_label)
            
            prediction = (video_file, video_pred_label, video_pred_score, video_true_score, true_label, segment_outputs)  # Ensure tuple has 6 elements
            predictions.append(prediction)
            save_partial_results(prediction, "results.json")
        except Exception as e:
            print(f"Error processing video {video_file}: {e}")
            continue
    return predictions



def save_partial_results(prediction, filename):
    video_file, video_pred_label, video_pred_score, video_true_score, video_true_label, segment_outputs = prediction
    video_result = {
        "video_file": video_file,
        "video_pred_label": video_pred_label,
        "video_pred_score": video_pred_score,
        "video_true_score": video_true_score,
        "video_true_label": video_true_label,
        "segments": []
    }
    for i, (segment_label, segment_score, probabilities) in enumerate(segment_outputs):
        segment_video_label_score = probabilities[0, video_pred_label].item()
        segment_true_label_score = probabilities[0, video_true_label].item()
        video_result["segments"].append({
            "segment_index": i + 1,
            "segment_label": segment_label,
            "segment_score": segment_score,
            "segment_video_label_score": segment_video_label_score,
            "segment_true_label_score": segment_true_label_score,
            # Assuming sv_true_label and sv_video_pred are not needed here or add them back if available in predictions
        })
    
    if os.path.exists(filename):
        with open(filename, "r+") as f:
            results = json.load(f)
            results.append(video_result)
            f.seek(0)
            json.dump(results, f, indent=4)
    else:
        with open(filename, "w") as f:
            json.dump([video_result], f, indent=4)


def load_existing_results(filename):
    if os.path.exists(filename):
        with open(filename, "r") as f:
            results = json.load(f)
            processed_files = {result["video_file"] for result in results}
            return results, processed_files
    return [], set()

def save_results(predictions, filename="results.json"):
    results = []
    for video_file, video_pred_label, video_pred_score, video_true_score, true_label, segment_outputs in predictions:
        video_result = {
            "video_file": video_file,
            "video_pred_label": video_pred_label,
            "video_pred_score": video_pred_score,
            "video_true_score": video_true_score,
            "video_true_label": true_label,
            "segments": []
        }
        for i, (segment_label, segment_score, probabilities) in enumerate(segment_outputs):
            segment_video_label_score = probabilities[0, video_pred_label].item()
            segment_true_label_score = probabilities[0, true_label].item()
            video_result["segments"].append({
                "segment_index": i + 1,
                "segment_label": segment_label,
                "segment_score": segment_score,
                "segment_video_label_score": segment_video_label_score,
                "segment_true_label_score": segment_true_label_score,
                # Assuming sv_true_label and sv_video_pred are not needed here or add them back if available in predictions
            })
        results.append(video_result)

    with open(filename, "w") as f:
        json.dump(results, f, indent=4)


def compute_metrics(predictions):
    true_labels = [pred[4] for pred in predictions]
    pred_labels = [pred[1] for pred in predictions]
    
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    
    return accuracy, precision, recall, f1

def save_performance_metrics(accuracy, precision, recall, f1, time_consumed, cpu_energy, gpu_energy, filename="performance.json"):
    performance = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "time_consumed": time_consumed,
        "cpu_energy": cpu_energy,
        "gpu_energy": gpu_energy
    }
    
    with open(filename, "w") as f:
        json.dump(performance, f, indent=4)

# Configuration
config = {
    "model_name": "facebook/timesformer-base-finetuned-k400",
    "image_processor_name": "MCG-NJU/videomae-base-finetuned-kinetics",
    "num_samples": 100,
    "num_classes": 30,  # For flexible dataset input
    "num_samples_per_class": 25,  # For flexible dataset input
    "video_list_path": "archive/kinetics400_val_list_videos.txt",
    "video_directory": "archive/videos_val",
    "use_exact": True
}

# Initialize processors
video_processor = VideoProcessor(config["model_name"], config["image_processor_name"])
shap_calculator = TemporalShap(num_samples=config["num_samples"])

# Load existing results
existing_results, processed_files = load_existing_results("results.json")

# Read video list and organize by categories if necessary
video_labels = defaultdict(list)
with open(config["video_list_path"], "r") as f:
    for line in f:
        name, label = line.strip().split()
        video_labels[int(label)].append(name)

# Prepare video samples for the configured number of classes and samples
sampled_files = []
true_labels = []
selected_classes = random.sample(list(video_labels.keys()), config["num_classes"])
for cls in selected_classes:
    sampled_files.extend(random.sample(video_labels[cls], config["num_samples_per_class"]))
    true_labels.extend([cls] * config["num_samples_per_class"])

# Filter unprocessed files
unprocessed_files = [f for f in sampled_files if f not in processed_files]
unprocessed_labels = [true_labels[sampled_files.index(f)] for f in unprocessed_files]

# Record start time and energy consumption
start_time = time.time()
initial_cpu_energy = get_cpu_energy()
initial_gpu_energy = get_gpu_energy()

# Process videos
video_data = process_videos(video_processor, shap_calculator, unprocessed_files, unprocessed_labels, use_exact=config["use_exact"], start_index=len(existing_results))

# Record end time and energy consumption
end_time = time.time()
final_cpu_energy = get_cpu_energy()
final_gpu_energy = get_gpu_energy()
time_consumed = end_time - start_time
cpu_energy_consumed = final_cpu_energy - initial_cpu_energy
gpu_energy_consumed = final_gpu_energy - initial_gpu_energy

# Combine existing results with new data
all_results = existing_results + video_data

# Save results
save_results(all_results)

# Compute and output metrics
accuracy, precision, recall, f1 = compute_metrics(all_results)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Save performance metrics
save_performance_metrics(accuracy, precision, recall, f1, time_consumed, cpu_energy_consumed, gpu_energy_consumed, filename="performance.json")


# Print detailed results
for video_file, video_pred_label, video_pred_score, video_true_score, true_label, segment_outputs, sv_true_label, sv_video_pred in all_results:
    print(f"Video: {video_file}, Overall Predicted Label = {video_pred_label}, Overall Prediction Score = {video_pred_score:.4f}, True Label = {true_label}, True Label Score = {video_true_score:.4f}")
    for i, (segment_label, segment_score, probabilities) in enumerate(segment_outputs):
        segment_video_label_score = probabilities[0, video_pred_label].item()
        segment_true_label_score = probabilities[0, true_label].item()
        print(f"  Segment {i+1}: Predicted Label = {segment_label}, Prediction Score = {segment_score:.4f}, Segment Video Label Score = {segment_video_label_score:.4f}, Segment True Label Score = {segment_true_label_score:.4f}, SV True Label = {sv_true_label[i]:.4f}, SV Predicted Label = {sv_video_pred[i]:.4f}")


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


Error reading CPU energy: 'int' object has no attribute 'power'
Error reading GPU energy: Command '['nvidia-smi', '--query-gpu=energy.draw', '--format=csv,noheader,nounits']' returned non-zero exit status 2.


Processing videos:   7%|6         | 50/748 [00:00<?, ?video/s]

/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)


Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Error processing segment: shape '[1, 10, 14, 8, 768]' is invalid for input of size 903168
Skipping video wL4EN5CBU9Q.mp4 due to empty segment outputs.
Error processing segment: shape '[1, 17, 14, 8, 768]' is invalid for input of size 1505280
Error processing segment: shape '[1, 17, 14, 8, 768]' is invalid for input of size 1505280
Error processing segment: shape '[1, 